In [1]:
# -*- coding: utf-8 -*-
import json
import cv2 as cv
import numpy as np
import collections
import random as rand

In [2]:
class KalmanPather:
    def __init__(self, initial_x, Q, r):
        self.pvel = 0
        self.vel = 0
        self.accel = 0
        
        self.dt = 1
        #state transition matrix
        self.A = np.array([
                        [1, 0, self.dt, 0],
                        [0, 1, 0, self.dt],
                        [0, 0, 1, 0],
                        [0, 0, 0, 1]])
        # Measurement Matrix (H)
        self.H = np.array([[1, 0, 0, 0],
                      [0, 1, 0, 0]])
    
        # Process Noise Covariance (Q)
        self.q = 1e-6
        #precomputed Q for white noise in 4 dimensions with dt = 1 and variance = 2.35
        self.Q = np.array([[0.06527778, 0.19583333, 0.39166667, 0.39166667], 
              [0.19583333, 0.5875    , 1.175     , 1.175     ], 
              [0.39166667, 1.175     , 2.35      , 2.35      ], 
              [0.39166667, 1.175     , 2.35      , 2.35     ]])
        #average this with analytically derived Q
        self.Q = (self.Q + np.array([[self.dt**4/4, 0, self.dt**3/2, 0],
                          [0, self.dt**4/4, 0, self.dt**3/2],
                          [self.dt**3/2, 0, self.dt**2, 0],
                          [0, self.dt**3/2, 0, self.dt**2]])) * 0.5 * self.q
    
        # Measurement Noise Covariance (R)
        self.r = r
        self.R = self.r * np.eye(2)
    
        # Error Covariance (P)
        self.P = np.eye(4) * 0.001 #TEST

        # Control Variable Matrix
        self.B = np.array([[self.dt**2/2,0],
                          [0,self.dt**2/2],
                          [self.dt,0],
                          [0,self.dt]])
        # Initial State (x)
        #self.x = np.array(np.append([initial_x[0],initial_x[1]], [0, 0]))  # Initial guess
        self.x = np.array([0,0,0,0])
        
    def predictu(self, u):
        self.x_pred = (self.A).dot(self.x) + (self.B).dot(u)
        self.P_pred = (self.A).dot((self.P).dot((self.A).T)) + self.Q
        self.pvel = self.vel
        self.vel = (self.x_pred - self.x)[:2]
        self.accel = self.vel - self.pvel
        
    def predict(self):
        self.x_pred = (self.A).dot(self.x)
        self.P_pred = (self.A).dot((self.P).dot((self.A).T)) + self.Q
        self.x = np.copy(self.x_pred) #TEST
        self.P = np.copy(self.P_pred) #TEST
        self.pvel = self.vel
        self.vel = (self.x_pred - self.x)[:2]
        self.accel = self.vel - self.pvel

    def update(self, z):
        self.IM = (self.H).dot(self.x_pred) 
        self.IS = (self.H).dot((self.P_pred).dot((self.H).T)) + self.R
        self.K = (self.P_pred).dot(((self.H).T).dot(np.linalg.inv(self.IS)))
        self.x_upd = self.x_pred + (self.K).dot(z - self.IM)
        self.P_upd = self.P_pred - (self.K).dot((self.IS).dot((self.K).T)) #new
        self.x = self.x_upd
        self.P = self.P_upd
        self.x_upd = self.bound(self.x_upd) #crash countermeasure?
        return self.x_upd, self.P_upd, self.K, self.IM, self.IS #TEST

    def bound(self, x):
        touch = False
        if x[0] < 0:
            touch = True
            x[0] = 0
            x[2] *= 0
        if x[0] > windx:
            touch = True
            x[0] = windx
            x[2] *= 0
        if x[1] < 0:
            touch = True
            x[1] = 0
            x[3] *= 0
        if x[1] > windy:
            touch = True
            x[1] = windy
            x[3] *= 0
        if touch:
            self.x = x
        return x

In [ ]:
windx = 2048
windy = 2048
img = np.zeros((windx,windy,3), np.uint8)

# Initialize lists to hold the tracks
observed_track = collections.deque(maxlen=20)
obs_acc_track = collections.deque(maxlen=20) #acceleration of mouse
#noisy_track = collections.deque(maxlen=20)
predicted_tracks = []

m = np.array((0,0))
z = np.array((0,0))
pm = m

def mousepath(event,cx,cy,flags,param):
    global pm
    global m
    #pm = m
    cv.line(img,(pm[0],pm[1]),(cx,cy),(255,255,255), 1, cv.LINE_AA)
    obs_acc_track.append(accel)
    m = np.array([cx,cy])
    #noisy_track.append((z[0],z[1]))
    
# Create a black image, a window and bind the function to window
cv.namedWindow('image')
cv.startWindowThread()
cv.setMouseCallback('image',mousepath)
img = np.zeros((windx,windy,3), np.uint8)

balls = []
cnt = 20
for i in range(cnt):
    q = 2 + i/(cnt/4)
    balls.append(KalmanPather((windx/2, windy/2), 10**(-1*q), 10**(-1*q)))
    predicted_tracks.append(collections.deque(maxlen=20))

pz = z
accel = z-pz
while(1):
    z = z + (m - z) / 10
    observed_track.append((z[0],z[1]))
    accel = (z-pz)/100
    print(balls[0].x)
    for i in range(len(balls)):
        #if (m.astype(int) == pm.astype(int)).any():
        if True:
            if i == 0: 
                balls[i].predictu(accel)
                x, P, K, IM, IS = balls[i].update(z)
            else:
                balls[i].predictu(balls[i-1].accel)
                x, P, K, IM, IS = balls[i].update(balls[i-1].x[:2])
                #x, P, K, IM, IS = balls[i].update(z)
        else:
            balls[i].predict()
            x = balls[i].x
            cv.circle(img, (int(windx/2), int(windy/2)), 50, (255,255,255), -1)

        predicted_tracks[i].append(x[:2].tolist())
        color = cv.cvtColor(np.uint8([[[179*((i+1)/len(balls)),255,255]]]),cv.COLOR_HSV2BGR)
        color = tuple(map(tuple, color[0]))[0]
        cv.circle(img, (int(x[:2][0].astype(int)), int(x[:2][1].astype(int))), 5, (int(color[0]),int(color[1]),int(color[2])), -1)

    
    cv.imshow('image',img)
    pz = z
    pm = m
    img = np.zeros((windx,windy,3), np.uint8)
    for q in range(len(balls)): 
        for i in range(1, len(predicted_tracks[q])):
            color = cv.cvtColor(np.uint8([[[179*((q+1)/len(balls)),255,255]]]),cv.COLOR_HSV2BGR)
            color = tuple(map(tuple, color[0]))[0]
            cv.line(img, tuple(map(int, predicted_tracks[q][i-1])), tuple(map(int, predicted_tracks[q][i])), (int(color[0]),int(color[1]),int(color[2])), 2, cv.LINE_AA)
        
    for i in range(1, len(observed_track)):
        cv.line(img, tuple(map(int, observed_track[i-1])), tuple(map(int, observed_track[i])), (255, 255, 255), 2, cv.LINE_AA)
    #for i in range(1, len(noisy_track)):
    #    cv.line(img, tuple(map(int, noisy_track[i-1])), tuple(map(int, noisy_track[i])), (150, 150, 150), 2, cv.LINE_AA)
    
    if cv.waitKey(20) == ord('q'):
        break

cv.waitKey(0)
cv.destroyAllWindows()
cv.waitkey(1)
cv.waitkey(1)

[0 0 0 0]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[29.30353155 19.71182897  4.27844602  2.91103488]
[77.68202145 52.3167077   9.79266528  6.69133962]
[134.24522958 100.11560933  15.14117786  11.67491606]
[195.28661648 157.87580061  20.19266082  16.88612259]
[258.47616391 221.65252715  24.81378315  21.9473539 ]
[322.2026869  288.68536148  28.94402906  26.66275788]
[385.32582532 357.01106656  32.56314411  30.9305542 ]
[447.01880908 425.20772541  35.6707022   34.70144581]
[506.66709613 492.22676407  38.27574029  37.95563133]
[563.80342492 557.28226376  40.39183741  40.69021224]
[618.06692464 619.77817621  42.03510953  42.91245386]
[669.17815411 679.26080969  43.22369621  44.63631818]
[716.92452053 735.38810399  43.97791869  45.88076592]
[761.15217271 787.90984758  44.32063374  46.66894262]
[801.7615787  836.65473984  44.27751623  47.02773041]
[838.70481302 881.52143002  43.87713691  46.9873

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
